## ODI Cricket Matches Dataset (1971 - 2023) Analysis

In [24]:
import psycopg2
import os
import pandas as pd

In [16]:
# Set environment variable for password 
os.environ['DB_PASSWORD'] = '*********'

conn = psycopg2.connect(host="localhost", database="CricketProject", user="postgres", password=os.environ['DB_PASSWORD'])

cur = conn.cursor()

In [17]:
cur.execute("SELECT * FROM batting LIMIT 10")
results = cur.fetchall()

for row in results:
    print(row)

(64381, 1, 'Sri Lanka', 1810, 5, 17, 0, 0, Decimal('29.41'), True, 'l')
(64357, 1, 'Pakistan', 1981, 35, 48, 2, 0, Decimal('72.91'), True, 'c')
(65455, 1, 'Australia', 1922, 16, 30, 1, 0, Decimal('53.33'), True, 'c')
(64381, 1, 'Sri Lanka', 1987, 6, 46, 0, 0, Decimal('13.04'), True, 'c')
(64357, 1, 'Pakistan', 1750, 114, 123, 14, 0, Decimal('92.68'), True, 'r')
(251495, 1, 'Australia', 2119, 24, 36, 5, 0, Decimal('66.66'), True, 'c')
(65455, 1, 'Australia', 1793, 9, 25, 1, 0, Decimal('36'), True, 'c')
(64555, 1, 'India', 1934, 1, 11, 0, 0, Decimal('9.09'), True, 'c')
(64381, 1, 'Sri Lanka', 1789, 37, 56, 3, 0, Decimal('66.07'), True, 'c')
(64357, 1, 'Pakistan', 1672, 65, 68, 5, 0, Decimal('95.58'), False, 'n')


In [21]:
cur.execute("SELECT * FROM batting LIMIT 5")
results = cur.fetchmany(5) 

df = pd.DataFrame(results)

print(df)

      0   1          2     3    4    5   6   7      8     9  10
0  64381   1  Sri Lanka  1810    5   17   0   0  29.41  True  l
1  64357   1   Pakistan  1981   35   48   2   0  72.91  True  c
2  65455   1  Australia  1922   16   30   1   0  53.33  True  c
3  64381   1  Sri Lanka  1987    6   46   0   0  13.04  True  c
4  64357   1   Pakistan  1750  114  123  14   0  92.68  True  r


In [ ]:
-- Whcih teams have won the most matches

SELECT match_winner as team, COUNT(*) as number_of_wins
FROM public.matches
group by match_winner
order by number_of_wins desc

In [ ]:
-- Winning percentage by team

with cte as (
SELECT match_winner as team, COUNT(*) as number_of_wins
FROM public.matches
group by match_winner
order by number_of_wins desc),

total_matches as (
SELECT team1_name AS teams
FROM public.matches
UNION ALL 
SELECT team2_name AS teams
FROM public.matches),

total as (
SELECT teams, count(*) as matches_played
FROM total_matches
group by teams)

select cte.team, cte.number_of_wins / total.matches_played :: float * 100 AS winning_percetage
from cte
join total 
	ON cte.team = total.teams
ORDER BY winning_percetage DESC;

In [ ]:
-- Players with the highest economy

SELECT p.player_name, AVG(b.economy) as avg_economy, count(*) as innings, sum(b.overs) as total_overs
FROM public.bowling as b
JOIN public.players as p
ON p.player_id = b.name
group by p.player_name
order by avg_economy desc

In [ ]:
-- Players with the lowest economy

SELECT p.player_name, AVG(b.economy) as avg_economy, count(*) as innings, sum(b.overs) as total_overs
FROM public.bowling as b
JOIN public.players as p
ON p.player_id = b.name
group by p.player_name
order by avg_economy asc;

In [ ]:
-- bowlers with most wickets 

SELECT  p.player_name, sum(b.wickets) as total_wickets, b.team
FROM public.bowling as b
JOIN public.players as p
ON p.player_id = b.name
group by p.player_name, b.team
order by total_wickets desc

In [ ]:
-- Players with the most runs scored

SELECT p.player_name, ba.team, SUM(ba.runs) as total_runs
FROM public.batting as ba
JOIN public.players as p
ON p.player_id = ba.batsman
group by p.player_name, ba.team
ORDER BY total_runs DESC;

In [ ]:
-- Players with the most runs conceded
SELECT p.player_name, b.team, SUM(b.conceded) as total_runs_conceded
FROM public.bowling as b
JOIN public.players as p
ON p.player_id = b.name
group by p.player_name, b.team
ORDER BY total_runs_conceded DESC

In [ ]:
-- Team captain with the most wins
with one as (
SELECT m.match_id,
CASE WHEN m.match_winner = m.team1_name THEN team1_captain END as ca
FROM public.matches as m),

two as (
SELECT m.match_id,
CASE WHEN m.match_winner = m.team2_name THEN team2_captain END as ca2
FROM public.matches as m),

three as (
SELECT one.match_id, COALESCE(ca,ca2) as captain
FROM one 
JOIN two 
ON one.match_id = two.match_id),

four as (
SELECT captain, p.player_name
FROM three
JOIN public.players as p
ON p.player_id = three.captain)

SELECT player_name, COUNT(*) AS wins_as_captain
FROM four
GROUP BY player_name
ORDER BY COUNT(*) DESC